In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!cp /content/drive/MyDrive/rossmann_dw.db /content/

!cp /content/drive/MyDrive/tree_predictions.csv /content/

!cp /content/drive/MyDrive/lstm_predictions.csv /content/

In [ ]:
import pandas as pd
from sqlalchemy import create_engine

# اتصال به دیتابیس
engine = create_engine('sqlite:///rossmann_dw.db')

# خواندن جدول‌ها
fact_sales = pd.read_sql('SELECT * FROM fact_sales', engine)
dim_store = pd.read_sql('SELECT * FROM dim_store', engine)
dim_date = pd.read_sql('SELECT * FROM dim_date', engine)

df = fact_sales.merge(dim_store, on='store_id', how='left')
df = df.merge(dim_date, on='date_id', how='left')

main_df = df.copy()
predictions = pd.read_csv('tree_predictions.csv')
lstm_predictions = pd.read_csv('lstm_predictions.csv')

predictions['preds_lstm'] = lstm_predictions['y_pred']
predictions['store_id'] = predictions['store_id'].apply(lambda x: x+1)

In [ ]:
import os
import re
import io
import json
from google.colab import userdata
from typing import Dict, Any, List, Optional, Tuple, Sequence

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import gradio as gr
import requests

# ================= CONFIG =================
LLM_PROVIDER = "groq"
GROQ_API_KEY = userdata.get("GROQ_API_KEY")
GROQ_MODEL = 'llama3-70b-8192'

predictions_df = predictions

REQUIRED_PRED_COLS = {"store_id", "date_id", "preds_lgb", "preds_xgb", "preds_lstm"}
MODEL_COLS = {"lgb": "preds_lgb", "xgb": "preds_xgb", "lstm": "preds_lstm"}

# ================= LLM (chat + router) =================
def llm_chat(messages: List[Dict[str, str]]) -> str:
    if not GROQ_API_KEY:
        return "(LLM offline)"
    try:
        url = "https://api.groq.com/openai/v1/chat/completions"
        headers = {"Authorization": f"Bearer {GROQ_API_KEY}", "Content-Type": "application/json"}
        payload = {"model": GROQ_MODEL, "messages": messages, "temperature": 0.2, "max_tokens": 512}
        r = requests.post(url, headers=headers, data=json.dumps(payload), timeout=60)
        r.raise_for_status()
        return r.json()["choices"][0]["message"]["content"]
    except Exception as e:
        return f"(LLM error) {e}"

def _extract_json(s: str) -> Optional[dict]:
    """Extract first JSON object from a string (handles extra prose/code fences)."""
    try:
        # Fast path
        return json.loads(s)
    except Exception:
        pass
    # Fallback: find first {...}
    try:
        m = re.search(r"\{.*\}", s, flags=re.S)
        if m:
            return json.loads(m.group(0))
    except Exception:
        return None
    return None

def _normalize_model(m: Optional[str]) -> Optional[str]:
    if not m:
        return None
    t = m.strip().lower()
    if any(x in t for x in ["lgb", "lightgbm", "lgbm"]): return "lgb"
    if any(x in t for x in ["xgb", "xgboost"]): return "xgb"
    if "lstm" in t: return "lstm"
    return None

def llm_route(user_text: str) -> Dict[str, Any]:
    """
    Ask the LLM to infer which action to run and which parameters to use.
    Returns a dict like:
      {"type": "...", "model": "...", "store": int, "k": int, "models": [...], "group_col": "..."}
    """
    system = {
        "role": "system",
        "content": (
            "You route user requests to analytics actions for Rossmann sales predictions. "
            "Always respond with STRICT JSON, no commentary. "
            "Schema keys: type (one of: next_month, next_week, compare_next_month, topk_next_week, group_next_month, freeform); "
            "model (lgb|xgb|lstm); models (array of models for comparison); store (int); k (int); group_col (string, like store_type/state/promo). "
            "If a needed field is missing, infer it. If genuinely unknown, omit it. "
            "For Top-K, default model=lgb if user didn't specify. "
            "Examples:\n"
            "{'type':'next_month','model':'lgb','store':1}\n"
            "{'type':'next_week','model':'xgb','store':20}\n"
            "{'type':'compare_next_month','models':['lgb','xgb','lstm'],'store':10}\n"
            "{'type':'topk_next_week','k':5,'model':'xgb'}\n"
            "{'type':'group_next_month','group_col':'store_type'}"
        )
    }
    user = {"role": "user", "content": f"User request: {user_text}\nReturn ONLY JSON."}
    raw = llm_chat([system, user])

    data = _extract_json(raw) or {}
    # Normalize/guard
    t = data.get("type") or "freeform"
    model = _normalize_model(data.get("model"))
    if not model and t == "topk_next_week":
        model = "lgb"  # default for Top-K if omitted
    # Normalize list of models for compare
    if data.get("models"):
        data["models"] = [m for m in ["lgb", "xgb", "lstm"] if _normalize_model(m) in {"lgb", "xgb", "lstm"} and m in [ _normalize_model(x) or "" for x in data["models"] ]]
        if not data["models"]:
            data["models"] = ["lgb", "xgb", "lstm"]
    # Clean group col: spaces -> underscores, lower
    if data.get("group_col"):
        data["group_col"] = re.sub(r"\s+", "_", str(data["group_col"]).strip().lower())

    # Coerce ints
    for key in ["store", "k"]:
        if key in data and data[key] is not None:
            try:
                data[key] = int(data[key])
            except Exception:
                data[key] = None

    return {
        "type": t,
        "model": model,
        "models": data.get("models"),
        "store": data.get("store"),
        "k": data.get("k"),
        "group_col": data.get("group_col"),
    }

# ================= PREPARE DATA =================
def _ensure_cols(df: pd.DataFrame, needed: Sequence[str], name: str):
    missing = set(needed) - set(df.columns)
    if missing:
        raise ValueError(f"{name} missing required columns: {sorted(missing)}")

_ensure_cols(predictions_df, REQUIRED_PRED_COLS, "predictions_df")
predictions_df = predictions_df.copy()
predictions_df["date_id"] = pd.to_datetime(predictions_df["date_id"])

# normalize main_df store id column to 'store_id' if it's 'Store'
main_df = main_df.copy()
if "store_id" not in main_df.columns and "Store" in main_df.columns:
    main_df = main_df.rename(columns={"Store": "store_id"})
if "store_id" not in main_df.columns:
    pass

# ================= PLOTTING HELPERS =================
def _save_fig_to_image(fig) -> Image.Image:
    buf = io.BytesIO()
    plt.tight_layout()
    fig.savefig(buf, format="png", dpi=160)
    plt.close(fig)
    buf.seek(0)
    return Image.open(buf)

def make_line_image(dates: pd.Series, values: pd.Series, title: str, ylabel: str = "Predicted Sales") -> Image.Image:
    fig, ax = plt.subplots(figsize=(8, 4.5))
    ax.plot(dates, values, marker="o")
    ax.set_title(title)
    ax.set_xlabel("Date")
    ax.set_ylabel(ylabel)
    ax.grid(True, linestyle="--", alpha=0.5)
    fig.autofmt_xdate()
    return _save_fig_to_image(fig)

def make_multi_line_image(dates: pd.Series, series_dict: Dict[str, pd.Series], title: str, ylabel: str = "Predicted Sales") -> Image.Image:
    fig, ax = plt.subplots(figsize=(8, 4.5))
    for label, vals in series_dict.items():
        ax.plot(dates, vals, marker="o", label=label)
    ax.set_title(title)
    ax.set_xlabel("Date")
    ax.set_ylabel(ylabel)
    ax.grid(True, linestyle="--", alpha=0.5)
    ax.legend()
    fig.autofmt_xdate()
    return _save_fig_to_image(fig)

def make_bar_image(categories: Sequence[str], values: Sequence[float], title: str, xlabel: str = "", ylabel: str = "Predicted Sales") -> Image.Image:
    fig, ax = plt.subplots(figsize=(8, 4.5))
    ax.bar(range(len(categories)), values)
    ax.set_title(title)
    ax.set_xticks(range(len(categories)))
    ax.set_xticklabels(categories, rotation=30, ha="right")
    ax.set_xlabel(xlabel)
    ax.set_ylabel(ylabel)
    ax.grid(axis="y", linestyle="--", alpha=0.5)
    return _save_fig_to_image(fig)

# ================= SHARED DATE HELPERS =================
def get_next_month_period(df: pd.DataFrame) -> pd.Period:
    """Pick the earliest month in the future; fallback to the latest month in df."""
    df = df.copy()
    df["ym"] = df["date_id"].dt.to_period("M")
    today = pd.Timestamp.today().normalize()
    future = df[df["date_id"] >= today]
    return (future["ym"].min() if not future.empty else df["ym"].max())

def get_next_7_dates(df: pd.DataFrame) -> List[pd.Timestamp]:
    """Earliest 7 unique dates in the future across the whole predictions_df; fallback to first 7 overall."""
    all_dates = df["date_id"].sort_values().unique()
    today = pd.Timestamp.today().normalize()
    future = [d for d in all_dates if d >= today]
    window = future[:7] if len(future) >= 7 else list(all_dates[:7])
    return list(pd.to_datetime(window))

# ================= HANDLERS =================
def handle_next_month(model: str, store: int) -> Tuple[str, Optional[Image.Image]]:
    df = predictions_df.query("store_id == @store").copy()
    if df.empty:
        return f"No predictions for store {store}.", None
    target_month = get_next_month_period(df)
    frame = (df.assign(month=df["date_id"].dt.to_period("M"))
               .loc[lambda d: d["month"] == target_month]
               .sort_values("date_id"))
    img = make_line_image(
        dates=frame["date_id"], values=frame[MODEL_COLS[model]],
        title=f"Store {store} — Next Month ({target_month}) — {model.upper()}"
    )
    total = float(frame[MODEL_COLS[model]].sum())
    return f"{model.upper()} forecast for store {store}, month {target_month}. Total: {total:,.2f}", img

def handle_next_week(model: str, store: int) -> Tuple[str, Optional[Image.Image]]:
    df = predictions_df.query("store_id == @store").sort_values("date_id")
    if df.empty:
        return f"No predictions for store {store}.", None
    week_dates = get_next_7_dates(predictions_df)
    week = df[df["date_id"].isin(week_dates)]
    if len(week) == 0:
        week = df.head(7)
    img = make_line_image(
        dates=week["date_id"], values=week[MODEL_COLS[model]],
        title=f"Store {store} — Next 7 Days — {model.upper()}"
    )
    total = float(week[MODEL_COLS[model]].sum())
    start = week["date_id"].min().date()
    end = week["date_id"].max().date()
    return f"{model.upper()} forecast for store {store} ({start} → {end}). Total: {total:,.2f}", img

def handle_compare_next_month(models: List[str], store: int) -> Tuple[str, Optional[Image.Image]]:
    df = predictions_df.query("store_id == @store").copy()
    if df.empty:
        return f"No predictions for store {store}.", None
    target_month = get_next_month_period(df)
    frame = (df.assign(month=df["date_id"].dt.to_period("M"))
               .loc[lambda d: d["month"] == target_month]
               .sort_values("date_id"))

    if frame.empty:
        return f"No next-month rows available for store {store}.", None

    series_dict = {}
    totals = []
    for m in models:
        col = MODEL_COLS[m]
        series_dict[m.upper()] = frame[col]
        totals.append((m.upper(), float(frame[col].sum())))

    img = make_multi_line_image(
        dates=frame["date_id"], series_dict=series_dict,
        title=f"Store {store} — Next Month ({target_month}) — Model Comparison"
    )
    totals_str = " | ".join([f"{name}: {val:,.2f}" for name, val in totals])
    return f"Monthly totals — {totals_str}", img

def handle_topk_next_week(k: int, model: str) -> Tuple[str, Optional[Image.Image]]:
    # Compute the same 7-day horizon for all stores
    week_dates = set(get_next_7_dates(predictions_df))
    df_week = predictions_df[predictions_df["date_id"].isin(week_dates)]
    if df_week.empty:
        # fallback: first 7 days overall
        first7 = predictions_df["date_id"].sort_values().unique()[:7]
        df_week = predictions_df[predictions_df["date_id"].isin(first7)]

    agg = (df_week.groupby("store_id", as_index=False)[MODEL_COLS[model]]
           .sum()
           .rename(columns={MODEL_COLS[model]: "pred_total"}))
    if agg.empty:
        return "No data available to rank stores.", None

    topk = agg.sort_values("pred_total", ascending=False).head(k)
    # Bar chart
    cats = [str(s) for s in topk["store_id"]]
    vals = [float(v) for v in topk["pred_total"]]
    img = make_bar_image(cats, vals, title=f"Top {k} Stores — Next 7 Days — {model.upper()}", xlabel="Store ID")

    # Text table
    lines = [f"{i+1}. Store {sid}: {v:,.2f}" for i, (sid, v) in enumerate(zip(topk["store_id"], topk["pred_total"]))]
    return "Leaderboard (sum of next 7 days):\n" + "\n".join(lines), img

def handle_group_next_month(group_col: str) -> Tuple[str, Optional[Image.Image]]:
    if "store_id" not in main_df.columns:
        return ("Aggregation needs main_df with a 'store_id' column "
                f"(found columns: {list(main_df.columns)[:8]}...)"), None
    if group_col not in main_df.columns:
        candidates = {c.lower(): c for c in main_df.columns}
        if group_col.lower() in candidates:
            group_col = candidates[group_col.lower()]
        else:
            return f"main_df does not have a '{group_col}' column.", None

    target_month = get_next_month_period(predictions_df)
    df_month = (predictions_df.assign(month=predictions_df["date_id"].dt.to_period("M"))
                              .loc[lambda d: d["month"] == target_month]
                              .copy())
    if df_month.empty:
        return "No rows for the next month horizon.", None

    # prepare store attributes (distinct per store)
    store_attrs = main_df[["store_id", group_col]].drop_duplicates(subset=["store_id"]).copy()
    # join attributes to predictions
    joined = df_month.merge(store_attrs, on="store_id", how="left")

    # if any stores missing attribute, label as "Unknown"
    joined[group_col] = joined[group_col].fillna("Unknown")

    model = "lgb"
    agg = joined.groupby(group_col, as_index=False)[MODEL_COLS[model]].sum()
    if agg.empty:
        return "No aggregated data available.", None

    cats = [str(x) for x in agg[group_col]]
    vals = [float(x) for x in agg[MODEL_COLS[model]]]
    img = make_bar_image(cats, vals, title=f"Next Month by {group_col} — {model.upper()}")
    # Top line text summary
    pairs = sorted(zip(cats, vals), key=lambda t: t[1], reverse=True)
    head = " | ".join([f"{c}: {v:,.0f}" for c, v in pairs[:5]])
    return f"Total predicted sales ({model.upper()}) for next month ({target_month}) by {group_col}.\nTop groups: {head}", img

# ================= ROUTER =================
def agent_router(user_text: str, chat_history: List[List[Any]]) -> Tuple[List[List[Any]], Optional[Image.Image]]:
    intent = llm_route(user_text)

    if intent["type"] == "next_month" and intent.get("model") and intent.get("store") is not None:
        msg, img = handle_next_month(intent["model"], int(intent["store"]))
        return chat_history + [[user_text, msg]], img

    if intent["type"] == "next_week" and intent.get("model") and intent.get("store") is not None:
        msg, img = handle_next_week(intent["model"], int(intent["store"]))
        return chat_history + [[user_text, msg]], img

    if intent["type"] == "compare_next_month" and intent.get("models") and intent.get("store") is not None:
        msg, img = handle_compare_next_month(intent["models"], int(intent["store"]))
        return chat_history + [[user_text, msg]], img

    if intent["type"] == "topk_next_week":
        k = intent.get("k") or 5
        model = intent.get("model") or "lgb"
        msg, img = handle_topk_next_week(int(k), model)
        return chat_history + [[user_text, msg]], img

    if intent["type"] == "group_next_month" and intent.get("group_col"):
        msg, img = handle_group_next_month(intent["group_col"])
        return chat_history + [[user_text, msg]], img

    system_prompt = (
        "You are a helpful retail analytics agent. "
        "When the user asks for predictions/analytics, prefer deterministic tools. "
        "For general questions, be concise and accurate."
    )
    messages = [{"role": "system", "content": system_prompt}]
    for u, a in (chat_history or [])[-5:]:
        if u: messages.append({"role": "user", "content": u})
        if a: messages.append({"role": "assistant", "content": a})
    messages.append({"role": "user", "content": user_text})
    reply = llm_chat(messages)
    return chat_history + [[user_text, reply]], None

# ================= UI =================
WELCOME_MSG = (
    "👋 Hi! I’m your Rossmann Sales Agent.\n\n"
    "**Things I can do:**\n"
    "• `predict next (month/week) sales with (lgb/xgb/lstm) model any for store`\n"
    "• `compare next month predictions by lgb, xgb, lstm for any store`\n"
    "• `show top k stores with highest predicted sales next week with those models`\n"
    "• `predict sales by store type next month`\n\n"
    "Ask for other views and I’ll try to help!"
)

with gr.Blocks(title="Rossmann Sales Agent", fill_height=True, theme=gr.themes.Soft()) as demo:
    gr.Markdown("# 🧠 Rossmann Sales Agent")
    chatbot = gr.Chatbot(value=[[None, WELCOME_MSG]], height=460, label="Conversation")
    user_in = gr.Textbox(placeholder="Type a request…", autofocus=True)
    send_btn = gr.Button("Send", variant="primary")
    plot_img = gr.Image(label="Plot", height=380)

    def on_send(user_text, history):
        history = history or [[None, WELCOME_MSG]]
        new_history, img = agent_router(user_text, history)
        return new_history, "", img

    send_btn.click(on_send, [user_in, chatbot], [chatbot, user_in, plot_img])
    user_in.submit(on_send, [user_in, chatbot], [chatbot, user_in, plot_img])

if __name__ == "__main__":
    demo.launch(debug=True)


/tmp/ipython-input-441424780.py:372: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot(value=[[None, WELCOME_MSG]], height=460, label="Conversation")


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://38c13ba03961eabb9a.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
